# Debugging tool to understand the behavior of pyproj.geod.fwd

In [2]:
import pyproj
from geobench.benchmark.dataset_converters import inspect_tools
from ipyleaflet import Map, Marker, Rectangle
import numpy as np
import rasterio

crs = pyproj.CRS.from_epsg(4326)

In [9]:
def leaflet_map(points, names):
    """Position all samples on a world map using ipyleaflet. Experimental feature."""
    # TODO need to use reproject to increse compatibility
    # https://github.com/jupyter-widgets/ipyleaflet/blob/master/examples/Numpy.ipynb

    map = Map(center=points[0], zoom=11)
    map.layout.height = "600"

    for point, name in zip(points, names):
        map.add_layer(Marker(location=point, draggable=False, opacity=0.5, title="allo"))

    return map


point_lat_lon = 45.630001, -73.519997 # lat, lon
img_shape = 100, 100
spatial_resolution = 10
radius_in_meter = spatial_resolution * img_shape[0] / 2


def get_rect(transform, shape):
    """Obtain a georeferenced rectangle ready to display in ipyleaflet."""
    sw = transform * (0, 0)
    ne = transform * shape
    return Rectangle(bounds=(sw[::-1], ne[::-1]))

def bounding_points_from_center(lat_center, lon_center, radius_in_meter):
    geod = pyproj.Geod(ellps='clrk66')
    lon, lat, baz = geod.fwd([lon_center]*4, [lat_center]*4, [0,90,180,270], [radius_in_meter]*4)

    return list(zip(lat, lon))

def center_to_transform(lat_center, lon_center, radius_in_meter, img_shape):
    geod = pyproj.Geod(ellps='clrk66')
    lon, lat, baz = geod.fwd([lon_center]*4, [lat_center]*4, [0,90,180,270], [radius_in_meter]*4)
    north, east, south, west = lat[0], lon[1], lat[2], lon[3]
    transform = rasterio.transform.from_bounds(west, south, east, north, *img_shape)
    return transform

def transform_to_center(transform, img_shape):
    lon_lat = transform * (np.array(img_shape) / 2.)
    return lon_lat


transform = center_to_transform(*point_lat_lon, radius_in_meter, img_shape)
print(transform_to_center(transform, img_shape))
points_lat_lon = bounding_points_from_center(*point_lat_lon, radius_in_meter)

map = leaflet_map( [point_lat_lon] + points_lat_lon, ['montreal']* 5)

map.add_layer(get_rect(transform=transform, shape=img_shape))
map



(-73.519997, 45.63000099820106)


Map(center=[45.630001, -73.519997], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…